## To Do

In [91]:
# To Do
# Organize the Cleaning code into readable groups
# Document the cleaning process
# Checking datatypes of both sets
# Considering renaming any columns (LOCATION, TIME)
# Checking for data loss thorughout the cleaning process

## Requirements

In [2]:
# Importing Dependencies
import pandas as pd
import os
import csv
import chardet

In [5]:
# The WHO Mortaity Dataset

# Combine Mortality rate files into a single dataframe
dirLoc = 'Resources' #filepath of the directory
Mortality_df = pd.DataFrame()

for file in os.scandir(dirLoc):
    if "Morti" in file.path: # Only append files from the Mortality Datasets
        Mortality_df = Mortality_df.append(pd.read_csv(file.path,)) #append the data of the current mortality file being read

c:\Users\daypa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\Users\daypa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
# Reading in the OECD Meat Consumption dataset
OECD_df = pd.read_csv('Resources/OECD_Meat_Consumption.csv')

## Data Exploration

Summary:
* Scope of the datasets (time)
* Irrelevant fields
* Irrelevant records and null values
* Data types
* Full Country Name Mapping

### Time range of dataset

In [16]:
# Investigating the time the datasets encompass

# The Mortaility rate dataset
Mortality_max = Mortality_df['Year'].max()
Mortality_min = Mortality_df['Year'].min()
print(f'The earliest year in the mortality dataset is {Mortality_min} and the latest year is {Mortality_max}')

OECD_max = OECD_df['TIME'].max()
OECD_min = OECD_df['TIME'].min()
print(f'The earliest year in the OECD dataset is {OECD_min} and the latest year is {OECD_max}')

# We want to narrow the scope of our records between the year 2000 and 2020
Mortality_df = Mortality_df[Mortality_df['Year'] >= 2000]

OECD_df = OECD_df[OECD_df['TIME'] >= 2000]
OECD_df = OECD_df[OECD_df['TIME'] <= 2020]

The earliest year in the mortality dataset is 2000 and the latest year is 2020
The earliest year in the OECD dataset is 2000 and the latest year is 2020


### Irrelevant Fields

In [38]:
# Limiting the field in the mortality dataset to: ["Country","Year","Cause","Sex","Deaths1"]
Mortality_df = Mortality_df[["Country","Year","Cause","Sex","Deaths1"]]

# Limiting the OECD dataset to the fields: ['LOCATION', 'SUBJECT', 'MEASURE','TIME','Value']
OECD_df = OECD_df[['LOCATION', 'SUBJECT', 'MEASURE','TIME','Value']]

Irrelevant Records

In [35]:
# Mortality Dataset
# Removing records with value 9
Mortality_df["Sex"].value_counts() # There are 20,701 records with the 9 value in the sex field which refers to other

# Creating new dataframe wihtout 9 values in the Sex field
Mortality_df = Mortality_df[Mortality_df["Sex"] != 9]

Mortality_df["Sex"].value_counts() # 9 values removed from the original dataset

print(Mortality_df.count()) # There are no null values

Country    4039092
Year       4039092
Cause      4039092
Sex        4039092
Deaths1    4039092
dtype: int64


In [ ]:
# OECD Dataset

OECD_df['MEASURE'].unique() # Dataset contains records in two units

OECD_df = OECD_df[OECD_df['MEASURE'] != 'THND_TONNE'] # Filtering out values measured in Thousand tonnes

OECD_df.count() # No null values present

LOCATION    3192
SUBJECT     3192
MEASURE     3192
TIME        3192
Value       3192
dtype: int64

### Data types

In [36]:
Mortality_df.dtypes # Columns Cause, Sex, and Country dont seem to be the datatype we need them to be

# We can explicitly assign the string datatype to the Cause column
Mortality_df['Cause'] = Mortality_df['Cause'].astype('string')
Mortality_df['Country'] = Mortality_df['Country'].astype('string')

Mortality_df.dtypes # The Cause Column is now string

Country    string
Year        int64
Cause      string
Sex         int64
Deaths1     int64
dtype: object

In [50]:
OECD_df.dtypes # LOCATION, SUBJECT, AND MEASURE should be string
OECD_type_dict = {'LOCATION':'string', 'SUBJECT':'string', 'MEASURE':'string'}
OECD_df = OECD_df.astype(OECD_type_dict)
OECD_df.dtypes

LOCATION     string
SUBJECT      string
MEASURE      string
TIME          int64
Value       float64
dtype: object

### Country Name Mapping

## WHO Country and Cause of Death Coding
The world health organization codes the causes of death and the country of origin using their mapping which is specified in tables in the Mortality dataset documentation.
I need to map the value of the codes to our clean dataset

In [101]:
# mapping the country name to the dataset using the WHO_Country_Map.csv file
# Read up on the .map() documentation in pandas

# Reading in the WHO country map as a dictionary


Code_To_Country = {}

with open('Resources/WHO_Country_Map.csv', mode = 'r') as file:
    reader = csv.reader(file)
    Code_To_Country = {rows[0] : rows[1] for rows in reader}


In [102]:
Code_To_Country
Code_To_Country.pop('country') # removing the header from the map


#apply the Code_To_Country map to the "Country" series in df_2000
df_2000['Country'] = df_2000['Country'].map(Code_To_Country)

<ipython-input-102-947fb4adf249>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2000['Country'] = df_2000['Country'].map(Code_To_Country)


In [111]:
OECD_df['LOCATION'].dtypes
OECD_df['LOCATION'].astype('string')


10       AUS
11       AUS
12       AUS
13       AUS
14       AUS
        ... 
12146    GBR
12147    GBR
12148    GBR
12149    GBR
12150    GBR
Name: LOCATION, Length: 3192, dtype: string

In [112]:
# Reading in the country code map
Codes_df = pd.read_csv('Resources/ISO_Alpha3_Country_Codes.csv')
Codes_df.head()

,ountry,Alpha-3 code
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA
3,American Samoa,ASM
4,Andorra,AND


In [113]:

ISOMap_File = 'Resources/ISO_Alpha3_Country_Codes.csv'
Alpha3_To_Country = {}

with open(ISOMap_File, 'rb') as chardetect:
    print(chardet.detect(chardetect.read()))  #Determine the encoding used in the file with chardet


with open('Resources/ISO_Alpha3_Country_Codes.csv', 'r', encoding = 'cp1252') as file:
    reader = csv.reader(file)
    Alpha3_To_Country = {row[1] : row[0] for row in reader } # read in the file using cp1252 encoding

    

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


In [114]:
Alpha3_To_Country.pop('Alpha-3 code') # remove the header from the dictionary

'ountry'

In [115]:
#Swap out the three letter codeswith the country name
OECD_df.head()
print(len(OECD_df['LOCATION'].unique()))

38


In [116]:
OECD_df['LOCATION'] = OECD_df['LOCATION'].map(Alpha3_To_Country)
OECD_df.rename(columns = {'LOCATION': 'Country','TIME':'Year'}, inplace=True)

In [117]:
len(OECD_df['Country'].unique())

36

In [121]:
# Joining the OECD and WHO Dataset
OECD_df #3192 records
df_2000 #4039092 records

lethalMeat_df = df_2000.merge(right=OECD_df , how='inner' , on= ['Country','Year'])

In [122]:
lethalMeat_df.head()

,Country,Year,Cause,Sex,Deaths1,INDICATOR,SUBJECT,MEASURE,Value
0,Egypt,2000,A00,1,2,MEATCONSUMP,BEEF,KG_CAP,7.700
1,Egypt,2000,A00,1,2,MEATCONSUMP,PIG,KG_CAP,0.035
2,Egypt,2000,A00,1,2,MEATCONSUMP,POULTRY,KG_CAP,7.494
3,Egypt,2000,A00,1,2,MEATCONSUMP,SHEEP,KG_CAP,1.586
4,Egypt,2000,A00,2,1,MEATCONSUMP,BEEF,KG_CAP,7.700


In [123]:
# replace NaN with values we can work with 'Missing'
len(lethalMeat_df)

5110360